In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import os

print(f"[API KEY]\n{os.environ['OPENAI_API_KEY']}")
print(os.environ["LANGCHAIN_TRACING_V2"])

[API KEY]
sk-proj-2ALiWBzcJl4s9ri6EUJ6T3BlbkFJxbUDIanlzbIf6JMPE7o2
true


In [3]:
import torch

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
# import fitz
from pdfminer.high_level import extract_text

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from transformers import AutoModel, AutoTokenizer

from langchain_community.chat_models import ChatOllama
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.callbacks.manager import CallbackManager


# EEVE 모델 사용
llm = ChatOllama(
    temperature=0.7,
    top_k=40,
    top_p=0.9,
    model="EEVE:latest",
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
)

In [8]:
import os
current_directory = os.getcwd()
print(current_directory)

/home/choi/Git/RAG_con_doc/langchain


In [5]:
# PDF 파일에서 텍스트 추출
text = extract_text("./data/LH_2016.pdf")

print(text)

20510  시공측량  및  규준틀

20510시공측량 및 규준틀

1.

일반사항

1.1

적용범위

1.1.1 요약

이 절은 당해 공사 시행을 위하여 수급인이 이행하여야 할 측량과 규준틀 설치의

일반적 요건에 관하여 적용한다.

1.1.2 주요내용

가.시공측량

나.공사기록

다.규준틀

라.지불과 검측을 위한 측량

1.2

관련  시방절

이 공사와 관련이 있는 사항 중 이 절에서 언급된 것 이외의 사항은 다음 절의 해

당사항에 따른다.

11510제출물 관리

13010준공

1.3

용어의  정의 

1.3.1 수준점(BenchMark,B.M)

“수준점”이라 함은 수준면에서의 높이를 정확히 구하여 놓은 점으로 고저측량의 기

준이 되는 점을 말한다.

1.3.2 가수준점(TemporaryBenchMark,T.B.M)

"임시수준점“이라 함은 건설공사 현장 또는 설계 구간 내에 임시로 설치한 수준점을

말하며,국가기준점과는 구분된다

1.3.3 인조점

“인조점”이라 함은 노선 측량 등에 있어서 중요한 측점에 박아진 말뚝을 보호하기

위함과 파손 또는 이동 등의 경우에 복원하기 위하여,조합할 목적으로 말뚝을 박는

점을 말한다.

1.4

제출물

다음 사항은 “11510제출물 관리”에 따라 제출하여야 한다.

1.4.1 착공 전 제출물(SD-1)

LH  전문시방서  /  2012.  6

20510  -  1

2.  공통공사

20510시공측량 및 규준틀

가.확인측량 작업 계획서

나.확인측량 결과 보고서

다.규준틀 설치 및 운영계획서

1.5

시공측량 

1.5.1 구비조건

가.수급인은 당해 공사의 정확한 시공을 위하여 공인된 측량기구를 항시 현장에 구비

하고 운영한다.

나.측량기술자는 측량법에 의하여 자격을 획득한 기술자이거나 이와 동등 이상의 측량

기술에 능통한 자로서 당해 공사 측량을 원활히 진행할 수 있는 인원을 배치하여야

한다.

1.5.2 확인측량 실시

가.수

In [9]:
# 전처리된 텍스트 파일 경로 설정
input_file = './data/LH_2016_no_enter.txt'

# 텍스트 파일 읽기
with open(input_file, 'r', encoding='utf-8') as file:
    text = file.read()

In [23]:
# 1단계 : 문서 분할
document = Document(page_content=text)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents([document])
print(f"분할된 청크의 수 : {len(split_documents)}")

분할된 청크의 수 : 338


In [73]:
print(split_documents[1].page_content)

가.수급인은 작업 착수 15일 전에 발주 설계도면과 실제 현장의 이상 유무를 확인하기
위하여 다음 각 호의 내용이 포함된 “확인측량 작업 계획서”를 2부 작성하여 감독자
의 승인을 받아야 한다.
1)측량의 방법
2)측량에 사용되는 기계,기구의 종류
3)측량에 참여할 기술자 조직도 및 명단
-측량 기술자의 이름,주소 및 전화번호 포함
4)측량 예정 일정표
5)측량의 정밀도
6)오차의 처리대책
나.수급인은 삼각점 또는 도근점에서 중간점(IP)등의 측량기준점 위치(좌표)를 확인한
다.
다.수급인은 기준점이 공사 시행 중 유실되는 것을 방지하기 위하여 필히 인조점을 설
치하여야 하며,인조점과 기준점 관계를 도면화하여 비치하여야 한다.
라.가수준점 설치
1)수급인은 공사 준공시까지 보존할 수 있는 가수준점을 시공에 편리한 위치에 설치
하고,공사기록문서에 좌표와 위치를 기록해야 한다.
2)가수준점은 사업지구 인근의 수준점으로부터 왕복 수준측량을 실시하여 허용오차
범위 이내이어야 한다.
3)가수준점 설치위치 및 측량 성과표 등은 감독자에게 보고하여 승인을 받은 후 사
용한다.
마.수급인은 인접공구 또는 기존 시설물과의 접속부 등의 이상 유무를 확인하여 이상
발견시 감독자에게 보고후 협의하여 적절한 처리대책을 강구한다.
바.수급인은 확인측량 완료시에는 다음 각 호의 내용이 포함된 “확인측량 결과 보고서”
2.  공통공사
20510  -  2
LH  전문시방서  /  2012.  6
20510시공측량 및 규준틀
를 2부 작성하여 감독자의 승인을 받아야 한다.
1)검토의견서
2)확인측량 결과 도면 (종․횡단도,평면도,구조물도 등)
3)가수준점 설치 위치 및 측량성과표
4)산출내역서
5)공사비 증감 대비표
6)기타 참고사항(측량 야장 사본 등)
사.수급인은 현지 확인측량결과 설계내용과 현저히 상이할 때는 감독자에게 측량결과
를 보고한 후 지시를 받아 시공에 착수하여야 한다.
1.5.3 시공측량
가.수급인은 정확한 공사를 위하여 해당공사 착수 전과 작업완료 후 측량을 실시하여


In [34]:
# 2단계 : 도큐먼트별로 ID 부여
from uuid import uuid4

for document in split_documents:
    document.metadata['id']=str(uuid4())

In [72]:
print(split_documents[3].metadata)

{'id': '28323f32-33b2-42ea-92b5-26ece9a565a3'}


In [79]:
# 단계 1: 퓨샷 프롬프트 생성(Create Few Shot Prompt)
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

examples = [
    {
        "document": "그리고 그가 말했다, '엄마, 저 왔어요.'",
        "sentence2": """그는 학교 버스가 그를 내려주자마자 엄마에게 전화를 걸었다.\tneutral
        그는 한마디도 하지 않았다.\tcontradiction
        그는 엄마에게 집에 갔다고 말했다.\tentailment
""",
    },
    {
        "sentence1": "내가 무엇을 위해 가고 있는지 또는 어떤 것을 위해 있는지 몰랐기 때문에 워싱턴의 지정된 장소에 보고해야 했습니다.",
        "sentence2": """나는 워싱턴에 가본 적이 없어서 거기 배정을 받았을 때 그 장소를 찾으려다가 길을 잃었어요.\tneutral
        워싱턴으로 진군하면서 해야 할 일이 무엇인지 정확히 알고 있었다.\tcontradiction
        나는 내가 무엇을 할 것인지 확신할 수 없어서 내가 보고를 하도록 배정받은 워싱턴으로 갔다.\tentailment
""",
    },
    {
        "sentence1": "그는 갈 수 없었다.",
        "sentence2": """그는 박물관 개관식에 가는 것이 허용되지 않았다.\tneutral
        그는 처음 초대를 받은 사람이고 그 경험을 즐겼다.\tcontradiction
        그는 참석하는 것을 허가받지 않았다.\tentailment
""",
    },
    {
        "sentence1": "그리고 저는 '괜찮아' 하고, 그게 다였어요!",
        "sentence2": """내가 좋다고 말했을 때, 우리는 그날 결혼하기로 결정했다.\tneutral
        나는 아니라고 대답했고, 그것은 계속 이어졌다.\tcontradiction
        내가 승낙한 후에, 그것은 끝났다.\tentailment
""",
    },
    {
        "sentence1": "저는, 그냥 알아내려고 거기 있었어요.",
        "sentence2": """나는 돈이 어디로 갔는지 이해하려고 했어요.\tneutral
        나는 처음부터 그것을 잘 이해했다.\tcontradiction
        이해하려고 노력하고 있었어요.\tentailment
""",
    },
]

example_prompt = PromptTemplate.from_template(
    '''You are a natural language understanding assistant tasked with creating sentences for a dataset. Your job is to read the given first sentence and then generate three different second sentences: one that is neutral, one that is a contradiction, and one that is an entailment.
    After creating each sentence, append 'entailment', 'neutral', or 'contradiction' to the end of each sentence.
    sentence1:\n{sentence1}\nsentence2:\n{sentence2}'''
)

print(example_prompt.format(**examples[0]))

In [80]:
# 단계 4: LLM 생성
llm = ChatOllama(model='EEVE:latest')
# llm = ChatOpenAI(model_name='gpt-4o')
# llm = ChatOllama(model='llama3_kor:latest')

In [81]:
# 단계 5 : chain 생성
chain = prompt_template | llm | StrOutputParser()

In [82]:
# 단계 6 : 
# for document in split_documents:
#     print(chain.invoke({"document":document.page_content}))
#     break
print(chain.invoke({"document":split_documents[0].page_content}))


After carefully reading and understanding the provided Korean document, I have extracted the following clear and relevant instructions for installing insulation materials:

1. Material Specifications: Use high-quality insulation material with a minimum R-value of 3.5 to ensure proper thermal resistance. Choose a suitable type of insulation based on the climate conditions and desired energy efficiency level.
2. Installation Methods: Properly install insulation materials according to the manufacturer's guidelines, ensuring that they are evenly distributed and have no gaps or voids. Use appropriate fasteners to secure the insulation in place, such as staples or adhesive tapes.
3. Safety Precautions: Wear proper safety gear, including gloves, eye protection, and a mask, when handling and installing insulation materials. Avoid inhaling fiberglass particles and follow all safety guidelines provided by the manufacturer.
4. Inspection: Regularly inspect the installed insulation for any signs o

In [24]:
# 문서 로드 및 임베딩
embeddings = HuggingFaceEmbeddings(model_name = 'jhgan/ko-sroberta-multitask')
# embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

# 검색기 생성
retriever = vectorstore.as_retriever()

In [33]:
# 질문에 대한 관련 문서 검색
query = "고로 슬래그의 굵은골재 기준에 대해 알려줘"
retrieved_docs = retriever.invoke(query)

# 검색된 문서 출력
for doc in retrieved_docs:
    # print(f"Retrieved Document: {doc.page_content}")
    # print(f"Retrieved Document: {doc.metadata}")
    print(doc.metadata)

{}
{}
{}
{}
